In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/siyuyang/Source/repos/GitHub_MSFT/CameraTraps-api')

In [3]:
import io
import os
import json

import numpy as np
import requests
from requests_toolbelt.multipart import decoder
from PIL import Image

from visualization.visualization_utils import draw_bounding_boxes_on_image, draw_bounding_box_on_image

In [4]:
# this example notebook will process the first 8 images with filenames ending in .jpg or .JPG in this directory
sample_input_dir = '../sample_input/test_images'

ip_address = ''
port = ''
base_url = 'http://{}:{}/v1/camera-trap/sync/'.format(ip_address, port)  # insert the IP address of the API
base_url

# override with production URL
base_url = 'http://boto.eastus.cloudapp.azure.com:6002/v1/camera-trap/sync/'
base_url

'http://:/v1/camera-trap/sync/'

'http://boto.eastus.cloudapp.azure.com:6002/v1/camera-trap/sync/'

In [5]:
# copy API key if testing production endpoint
API_KEY = ''

## Find out the version of the camera trap model used
Good endpoint to check that it is alive

In [6]:
# re-using the headers field messes with the requests connection...
headers = {
    'Ocp-Apim-Subscription-Key': API_KEY
}

version_info = requests.get(base_url + 'detector_model_version', headers=headers)
version_info.text

'v4.1.0'

## Calling the API

In [7]:
%%time

# re-using the headers field messes with the requests connection...
headers = {
    'Ocp-Apim-Subscription-Key': API_KEY
}

num_images_to_upload = 8  # the /detect endpoint currently accepts a maximum of 8 images

detection_confidence = 0.8  # a value from 0 to 1
render_boxes = True  # True to have returned result contain annotated image files; in any case a json with result will be returned

params = {
    'confidence': detection_confidence,
    'render': render_boxes
}

files = {}

num_images = 0
open_files = []
images_processed = {}
for i, image_name in enumerate(sorted(os.listdir(sample_input_dir))):
    if not image_name.lower().endswith('.jpg'):
        continue
    
    if num_images >= num_images_to_upload:
        break
    else:
        num_images += 1
    
    img_path = os.path.join(sample_input_dir, image_name)
    fd = open(img_path, 'rb')
    open_files.append(fd)
    files[image_name] = (image_name, fd, 'image/jpeg')
    images_processed[image_name] = img_path  # check bbox rendering

print('number of images to send:', len(files))

number of images to send: 8
CPU times: user 618 µs, sys: 1.18 ms, total: 1.8 ms
Wall time: 2.16 ms


In [8]:
r = requests.post(base_url + 'detect', 
                  params=params,
                  files=files, headers=headers)
print('response: ', r.status_code)

if not r.ok:
    r.reason
    r.text
r.elapsed.total_seconds()

for fd in open_files:
    fd.close()

response:  200


8.286973

In [9]:
r.headers

{'Server': 'gunicorn/20.0.4', 'Date': 'Wed, 22 Jul 2020 21:18:45 GMT', 'Connection': 'close', 'Content-Type': 'multipart/form-data; boundary=3bc2e5933dfe44ce8376378cf6c64cb5', 'Content-Length': '5110729'}

## Reading the returned result

In [10]:
res = decoder.MultipartDecoder.from_response(r)

In [11]:
results = {}
images = {}

for part in res.parts:
    # part is a BodyPart object with b'Content-Type', and b'Content-Disposition', the later includes 'name' and 'filename' info
    headers = {}
    for k, v in part.headers.items():
        headers[k.decode(part.encoding)] = v.decode(part.encoding)

    if headers.get('Content-Type', None) == 'image/jpeg':
        #images[part.headers['filename']] = part.content
        c = headers.get('Content-Disposition')
        image_name = c.split('name="')[1].split('"')[0]  # somehow all the filename and name info is all in one string with no obvious forma
        image = Image.open(io.BytesIO(part.content))
        images[image_name] = image
    elif headers.get('Content-Type', None) == 'application/json':
        content_disposition = headers.get('Content-Disposition', '')
        if 'detection_result' in content_disposition:
            results['detection_result'] = json.loads(part.content.decode())
        elif 'classification_result' in content_disposition:
            results['classification_result'] = json.loads(part.content.decode())

In [12]:
results

{'detection_result': {'S1_D04_R6_PICT0020.JPG': [[0,
    0.006578,
    0.9401,
    0.9797779999999999,
    0.998,
    1]],
  'S1_D04_R6_PICT0021.JPG': [[0, 0.01237, 0.9344, 0.93057, 0.998, 1]],
  'S1_D04_R6_PICT0022.JPG': [[0, 0, 0.9251, 0.9905, 0.999, 1]],
  'S1_D04_R6_PICT0128.JPG': [[0.5913, 0.0171, 0.67462, 0.1704, 0.997, 1]],
  'S1_D04_R6_PICT0129.JPG': [],
  'S1_D04_R6_PICT0130.JPG': [],
  'S1_D04_R6_PICT0221.JPG': [[0.5308,
    0.9349,
    0.62034,
    0.9992099999999999,
    0.988,
    1]],
  'S1_D04_R6_PICT0222.JPG': [[0.5113, 0.8766, 0.6167, 0.9989, 0.999, 1]]},
 'classification_result': {}}

In [ ]:
for img_name, img in sorted(images.items()):
    print(img_name)
    img
    img.close()  # close after displaying
    print()

In [ ]:
# TO FIX draw_bounding_boxes_on_image() does not modify in-place...?

for image_name, image_path in images_processed.items():
    image = Image.open(image_path)
    detections = results['detection_result'][image_name]
    
    boxes, classes = [], []
    for i in detections:
        boxes.append(i[:4])
        classes.append(i[5])
    
    for box in boxes:
        draw_bounding_box_on_image(image,
                               box[0],
                               box[1],
                               box[2],
                               box[3],
                               clss=1,
                               thickness=4,
                               expansion=0,
                               display_str_list=(),
                               use_normalized_coordinates=True,
                               label_font_size=16)
        image